In [3]:
from IMPORTS import *

In [6]:
a = [1,2]

In [9]:
b = np.array(a)
b.shape

(2,)

In [10]:
c = np.ndarray(a)
c.shape

(1, 2)

In [5]:
a

[1, 2, 3]

In [14]:
d = np.mean(a[-10:])
d

1.5

In [14]:
env_id = 'CartPole-v1'
env = gym.make(env_id)
obs = env.reset()
obs

array([ 1.08632864e-02, -3.04061511e-02, -3.11447187e-02, -4.99648067e-06])

In [15]:
obs.astype('double')

array([ 1.08632864e-02, -3.04061511e-02, -3.11447187e-02, -4.99648067e-06])

In [16]:
torch.tensor(obs, dtype=torch.double)

tensor([ 1.0863e-02, -3.0406e-02, -3.1145e-02, -4.9965e-06],
       dtype=torch.float64)

In [12]:
a = [1.0,2,3,4,5,6,1,1,1,]

In [13]:
torch.tensor(a).std()

tensor(1.9365)